In [1]:
# using sqlite3 for data exploration + visualization. pandas - for df
import sqlite3
import pandas as pd

In [2]:
# ipython-sql for sql magic statements, matplotlib and seaborn for visualization 
!pip install ipython-sql
!pip install seaborn 
import seaborn as sns
%load_ext sql
import matplotlib.pyplot as plt


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# this either creates a new database or uses the existing one if it exists. it's done in thee current directory
# here we make a new database - netflix.db
conn = sqlite3.connect('netflix.db')
cur = conn.cursor()

In [4]:
# check the current directory. That's where the database is stored
%pwd

'C:\\Users\\haykg\\Documents\\github\\Data-wrangling'

In [5]:
# this is to be able to use the netflix database with magic sql commands
%sql sqlite:///netflix.db

In [6]:
df = pd.read_csv('netflix_titles.csv')

In [7]:
# pandas method that takes the pandas DataFrame df and writes it into my SQL database and names Chicago_socioeconomic_data.
# index = FALSE says **Do not write the DataFrame’s index** as a column in the SQL table. By default, `to_sql` will create 
# a column for the DataFrame’s index; setting `index=False` skips that.
# method = "multi" will allow to insert several rows simultaneously. Otherwise it would just be one row at a time.
df.to_sql("netflix", conn, if_exists="replace", index = False)

8807

In [8]:
# For below you can also
# Install the 'ipython-sql' and 'prettytable' libraries using pip
!pip install ipython-sql prettytable

# Import the 'prettytable' library, which is used to display data in a formatted table
import prettytable

# Set the default display format for prettytable to 'DEFAULT' (i.e., a simple table format)
prettytable.DEFAULT = 'DEFAULT'


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
%sql select * from netflix limit 5

 * sqlite:///netflix.db
Done.


show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,None,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable."
s2,TV Show,Blood & Water,None,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Maake Ka-Ncube, Odwa Gwanya, Mekaila Mathys, Sandi Schultz, Duane Williams, Shamilla Miller, Patrick Mofokeng",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth."
s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabiha Akkari, Sofia Lesaffre, Salim Kechiouche, Noureddine Farihi, Geert Van Rampelberg, Bakary Diombera",None,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Action & Adventure","To protect his family from a powerful drug lord, skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war."
s4,TV Show,Jailbirds New Orleans,None,None,None,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down among the incarcerated women at the Orleans Justice Center in New Orleans on this gritty reality series."
s5,TV Show,Kota Factory,None,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam Khan, Ahsaas Channa, Revathi Pillai, Urvi Singh, Arun Kumar",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV Comedies","In a city of coaching centers known to train India’s finest collegiate minds, an earnest but unexceptional student and his friends navigate campus life."


In [10]:
# checking the promary keys. We find there are no primary keys - so I will delete the useless "False" column and make show_id the main one.
%sql PRAGMA table_info(netflix)

 * sqlite:///netflix.db
Done.


cid,name,type,notnull,dflt_value,pk
0,show_id,TEXT,0,None,0
1,type,TEXT,0,None,0
2,title,TEXT,0,None,0
3,director,TEXT,0,None,0
4,cast,TEXT,0,None,0
5,country,TEXT,0,None,0
6,date_added,TEXT,0,None,0
7,release_year,INTEGER,0,None,0
8,rating,TEXT,0,None,0
9,duration,TEXT,0,None,0


In [11]:
# show_id is supposed to be the unique identifier - hence we need to check if there are any repeating values
%sql select show_id, count(*) from netflix group by show_id having count(*) > 1 limit 5

# no duplicates

 * sqlite:///netflix.db
Done.


show_id,count(*)


In [12]:
%%sql
--# Checking for nulls

CREATE TABLE if not exists null_table as
SELECT 'type' as column,         SUM(`type`       IS NULL) as null_count FROM netflix
  UNION ALL SELECT 'cast',         SUM(`cast`       IS NULL) FROM netflix
  UNION ALL SELECT 'show_id',      SUM(show_id      IS NULL) FROM netflix
  UNION ALL SELECT 'title',        SUM(title        IS NULL) FROM netflix
  UNION ALL SELECT 'director',     SUM(director     IS NULL) FROM netflix
  UNION ALL SELECT 'country',      SUM(country      IS NULL) FROM netflix
  UNION ALL SELECT 'date_added',   SUM(date_added   IS NULL) FROM netflix
  UNION ALL SELECT 'release_year', SUM(release_year IS NULL) FROM netflix
  UNION ALL SELECT 'rating',       SUM(rating       IS NULL) FROM netflix
  UNION ALL SELECT 'duration',     SUM(duration     IS NULL) FROM netflix
  UNION ALL SELECT 'listed_in',    SUM(listed_in    IS NULL) FROM netflix
  UNION ALL SELECT 'description',  SUM(description  IS NULL) FROM netflix;

select * from null_table where null_count > 0 order by null_count DESC



 * sqlite:///netflix.db
Done.
Done.


column,null_count
director,2634
country,831
cast,825
date_added,10
rating,4
duration,3


In [13]:
%%sql
--# to find out if some directors are likely to work with some cast to update the director names where possible    

with directs(movie, direct_cast) AS
(select title, concat(director, '---', `cast`) from netflix)
select direct_cast, count(*) as instances from directs 
group by direct_cast having instances > 1 order by count(*) DESC limit 100              

 * sqlite:///netflix.db
Done.


direct_cast,instances
---,352
---David Attenborough,15
"Rajiv Chilaka---Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, Rajesh Kava, Mousam, Swapnil",12
"---David Spade, London Hughes, Fortune Feimster",6
"---Michela Luci, Jamie Watson, Eric Peterson, Anna Claire Bartlam, Nicolas Aqui, Cory Doran, Julie Lemieux, Derek McGrath",5
"S.S. Rajamouli---Prabhas, Rana Daggubati, Anushka Shetty, Tamannaah Bhatia, Sathyaraj, Nassar, Ramya Krishnan, Sudeep",4
"Rathindran R Prasad---Aishwarya Rajesh, Vidhu, Surya Ganapathy, Madhuri, Pavel Navageethan, Avantika Vandanapu",4
Hernán Zin---,4
"---Michela Luci, Jamie Watson, Anna Claire Bartlam, Dante Zee, Eric Peterson",4
Walter C. Miller---Sam Kinison,3


In [14]:
%%sql
--# Let's look into the most common correspondances and check if there are cases with missing directors
--# then we will update the missing ones where the correspondance is sure
select title, director, `cast` from netflix where `cast` like '%Vatsal Dubey, Julie Tejwani, Rupa Bhimani%'

 * sqlite:///netflix.db
Done.


title,director,cast
Chhota Bheem,None,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, Rajesh Kava, Mousam, Swapnil"
Chhota Bheem - Neeli Pahaadi,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, Rajesh Kava, Mousam, Swapnil"
Chhota Bheem & Ganesh,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, Rajesh Kava, Mousam, Swapnil"
Chhota Bheem & Krishna: Mayanagari,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, Rajesh Kava, Mousam, Swapnil"
Chhota Bheem & Krishna: Pataliputra- City of the Dead,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, Rajesh Kava, Arun Shekher, Sanchit Wartak"
Chhota Bheem And The Broken Amulet,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, Rajesh Kava, Mousam, Swapnil"
Chhota Bheem And The Crown of Valhalla,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, Rajesh Kava, Mousam, Swapnil"
Chhota Bheem and the Incan Adventure,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, Rajesh Kava, Mousam, Swapnil"
Chhota Bheem and The ShiNobi Secret,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, Rajesh Kava, Mousam, Swapnil"
Chhota Bheem Aur Hanuman,Rajiv Chilaka,"Vatsal Dubey, Julie Tejwani, Rupa Bhimani, Jigna Bhardwaj, Rajesh Kava, Mousam, Swapnil"


In [15]:
# one of those cases where exact same cast has the same director. so we can update a few nulls
%sql update netflix SET director ='Rajiv Chilaka' where `cast` like '%Vatsal Dubey, Julie Tejwani%' and director is null

 * sqlite:///netflix.db
2 rows affected.


[]

In [16]:
# the other cases of director - cast don't look connected so we update the other directors to "not known" below
%sql update netflix set director = 'Not Given' where director is null;
# to make sure no longer any null values for directors
%sql select count(*) from netflix where director is null

 * sqlite:///netflix.db
2632 rows affected.
 * sqlite:///netflix.db
Done.


count(*)
0


In [107]:
%sql PRAGMA table_info(netflix)

 * sqlite:///netflix.db
Done.


cid,name,type,notnull,dflt_value,pk
0,show_id,TEXT,0,None,0
1,type,TEXT,0,None,0
2,title,TEXT,0,None,0
3,director,TEXT,0,None,0
4,country,TEXT,0,None,0
5,date_added,TEXT,0,None,0
6,release_year,INTEGER,0,None,0
7,rating,TEXT,0,None,0
8,duration,TEXT,0,None,0
9,listed_in,TEXT,0,None,0


## cleaning nulls of countries
(i) We will check the directors, county couples with the number of times they used the country as a location. <br>
(ii) Then we will order the country_numbers for each director from 1 to ..., with 1 being the most common country for a dir. <br>
(iii) Then we will only choose the cases where director filmed in no1 country at least twice. <br>
(iv) Afterwards, we will update the nulls of countries corresponding to the directors that have a favorite country.

In [76]:
%%sql
--# below a table is created with the corresponding most often used countries
drop table if exists countries_to_update;
create table countries_to_update as
with counts_direct as (
  SELECT
    director,
    country,
    COUNT(*) AS country_count
  FROM
    netflix
  WHERE
    director IS NOT NULL AND country IS NOT NULL and director !='Not Given'
  GROUP BY
    director, country order by country_count desc),
    
ranked_counts as (
    select director, country, country_count, 
row_number() over(partition by director order by country_count) as country_rank 
from counts_direct order by country_count DESC
)

select director, country as most_used_country from ranked_counts where country_rank = 1 and country_count > 1

 * sqlite:///netflix.db
Done.
Done.


[]

In [77]:
%%sql
--# using the new table 'most_used_country' to update the countries in netflix
update netflix as n
set country = ctu.most_used_country 
from countries_to_update as ctu
where n.director = ctu.director and n.country is null

 * sqlite:///netflix.db
0 rows affected.


[]

In [78]:
%%sql
--# for the rest of the countries we put 'Not Given'
update netflix set country = "Not Given" where country is Null

 * sqlite:///netflix.db
0 rows affected.


[]

## dealing with the rest of the nulls
(i) As cast is irrelevant for the analysis, we will delete the column. <br>
(ii) Next we will drop the rows for 'delete_added', 'rating', 'duration' because those are very few and won't affect analysis.

In [106]:
%%sql 
--# remove the cast and description
alter table netflix drop `description`;
alter table netflix drop `cast`;

 * sqlite:///netflix.db
(sqlite3.OperationalError) no such column: "`description`"
[SQL: --# remove the cast and description
alter table netflix drop `description`;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [110]:
%%sql
delete from netflix
where date_added is null or rating is null or duration is null

 * sqlite:///netflix.db
17 rows affected.


[]

In [112]:
%%sql
--# checking and seeing that there are no more nulls left
SELECT `type` as column,         SUM(`type`       IS NULL) as null_count FROM netflix
  UNION ALL SELECT 'show_id',      SUM(show_id      IS NULL) FROM netflix
  UNION ALL SELECT 'title',        SUM(title        IS NULL) FROM netflix
  UNION ALL SELECT 'director',     SUM(director     IS NULL) FROM netflix
  UNION ALL SELECT 'country',      SUM(country      IS NULL) FROM netflix
  UNION ALL SELECT 'date_added',   SUM(date_added   IS NULL) FROM netflix
  UNION ALL SELECT 'release_year', SUM(release_year IS NULL) FROM netflix
  UNION ALL SELECT 'rating',       SUM(rating       IS NULL) FROM netflix
  UNION ALL SELECT 'duration',     SUM(duration     IS NULL) FROM netflix
  UNION ALL SELECT 'listed_in',    SUM(listed_in    IS NULL) FROM netflix;

 * sqlite:///netflix.db
Done.


column,null_count
Movie,0
show_id,0
title,0
director,0
country,0
date_added,0
release_year,0
rating,0
duration,0
listed_in,0


In [74]:
%%sql
--# updating the rest of the countries as `Not Given`


 * sqlite:///netflix.db
778 rows affected.


[]